### Import and setup

In [1]:
import openai
from rich import print
from getpass import getpass

oai_api_key = getpass()

In [2]:
client = openai.OpenAI(api_key=oai_api_key)

### Tool Calling with `instructor`

In [5]:
import instructor

ins_client = instructor.from_openai(
    client, mode=instructor.Mode.RESPONSES_TOOLS
)

Now, `tool_calling` is a very simple technique. 

**It's like this:**
1. 👤 **You ask** something the AI doesn't know right now
2. 🤖 **AI realizes** "I need help with this!"
3. 🔧 **AI grabs the right tool** (like a web search app)
4. 🌤️ **Tool gives the answer** back to AI
5. 🤖 **AI tells you** in a nice way!

Let's go to OpenAI and check tool calling in action

---

How does it translate to code?

(Using the stock example in `instructor`)

In [26]:
from pydantic import BaseModel

class Citation(BaseModel):
    id: int
    url: str


class Summary(BaseModel):
    citations: list[Citation]
    summary: str


response = ins_client.responses.create(
    model="gpt-4.1-mini",
    input="Who won the Test world cup in 2025?",
    tools=[{"type": "web_search_preview"}],
    response_model=Summary,
)

15:20:59.293 POST api.openai.com/v1/responses
15:21:01.499 Reading response body


In [27]:
print(response)

Summary(
    citations=[
        Citation(id=1, url='https://www.icc-cricket.com/test-world-cup-2025/results'),
        Citation(id=2, url='https://en.wikipedia.org/wiki/2025_ICC_Test_Championship')
    ],
    summary='The 2025 ICC Test World Cup winner is determined based on the ICC Test Championship final results held
in 2025.'
)

In [17]:
logfire_token = getpass()

In [18]:
import logfire

logfire.configure(token=logfire_token)
logfire.instrument_pydantic_ai()
logfire.instrument_httpx(capture_all=True)

Logfire project URL: ]8;id=384356;https://logfire-us.pydantic.dev/pratos/build-w-ai\https://logfire-us.pydantic.dev/pratos/build-w-ai]8;;\
Currently retrying 1 failed export(s)
Currently retrying 1 failed export(s)


In [19]:
response = ins_client.responses.create(
    model="gpt-4.1-mini",
    input="What are the LLMs that have tool calling capabilities in the market?",
    tools=[{"type": "web_search_preview"}],
    response_model=Summary,
)

10:44:45.163 POST api.openai.com/v1/responses
10:44:51.396 Reading response body


In [20]:
print(response)

Summary(
    citations=[
        Citation(id=1, url='https://ai.googleblog.com/2023/05/introducing-bard-google-ai-search-experience.html'),
        Citation(id=2, url='https://openai.com/blog/gpt-4-api-announcement'),
        Citation(
            id=3,
            url='https://www.microsoft.com/en-us/research/blog/how-to-build-ai-applications-with-openai-and-azure-c
ognitive-services/'
        ),
        Citation(id=4, url='https://huggingface.co/blog/toolformer'),
        Citation(id=5, url='https://www.anthropic.com/index/introducing-claude')
    ],
    summary="Several large language models (LLMs) currently available in the market support tool calling 
capabilities. These include Google Bard, which integrates external tools to enhance search and information 
retrieval; OpenAI's GPT-4, which supports API-based tool integrations; Microsoft's AI models on Azure that combine 
LLMs with cognitive services for extended tool functionalities; Meta's Toolformer model that is designed to use 
external tools effectively; and Anthropic's Claude, which incorporates safe tool usage features. These LLMs enable 
enhanced interactivity by calling external APIs, plugins, or services to perform complex tasks and retrieve 
real-time information."
)

In [23]:
response = ins_client.responses.create(
    model="gpt-4.1-mini",
    input="What is the current time in Mumbai?",
    # tools=[{"type": "web_search_preview"}],
    response_model=Summary,
)

print(response)

11:09:02.487 POST api.openai.com/v1/responses
11:09:06.072 Reading response body


Summary(
    citations=[Citation(id=1, url='https://www.timeanddate.com/worldclock/india/mumbai')],
    summary='To know the current time in Mumbai, India, you can check a reliable world clock or time zone website 
such as timeanddate.com, which provides real-time updates for cities around the globe.'
)

In [24]:
class TimeInfo(BaseModel):
    time: str
    timezone: str


response = ins_client.responses.create(
    model="gpt-4.1-mini",
    input="What is the current time in Mumbai?",
    response_model=TimeInfo,
)

11:09:41.945 POST api.openai.com/v1/responses
11:10:11.076 Reading response body


In [25]:
response = ins_client.responses.create(
    model="gpt-4.1-mini",
    input="What is the current time in Mumbai?",
    tools=[{"type": "web_search_preview"}],
    response_model=Summary,
)

11:10:11.270 POST api.openai.com/v1/responses
11:10:19.314 Reading response body


---

We won't go into the OpenAI SDK code for tool calling, the json spec to read is confusing for a first timer.

What about external tools? 

- `instructor` sadly doesn't provide any good way to solve that.

There are other frameworks that let you build out stuff.

### Building agents with `pydantic-ai

Why `pydantic-ai`?

- `instructor` doesn't have a good api around custom tool calls. It does one thing well and you could mix and match with newer "agentic frameworks".
- `langchain` or `llamaindex` are much much better in terms of features and support.
- But there's still some learning curve and the API is frankly not my taste (my hot take).
- `pydantic-ai` offers the right balance where I can explain the fundamentals and we can grow out from there.
- There's other libraries as well -> `agno`, `marvin`, etc. You can choose whatever you need to. Too many to try out!

In [28]:
from pydantic_ai import Agent
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider

number = 102001

model = OpenAIModel("gpt-4o-mini", provider=OpenAIProvider(api_key=oai_api_key))
agent = Agent(
    model,
    system_prompt="Be concise, reply with one sentence.",
)

prompt = f"Can you find me square root of {number}?"

result = await agent.run(prompt)
print(result.output)

15:22:47.441 agent run
15:22:47.442   chat gpt-4o-mini
15:22:47.445     POST api.openai.com/v1/chat/completions
15:22:48.818 Reading response body


The square root of 102001 is 319.

In [29]:
from pydantic import BaseModel
from typing import Optional
from pydantic_ai import Agent


class LocationInfo(BaseModel):
    place: str
    temperature: Optional[float]
    # time: Optional[str]

In [30]:
agent = Agent(
    model,
    instructions=("Be concise,reply with one sentence"),
    deps_type=Deps,
    output_type=LocationInfo,
)

NameError: name 'Deps' is not defined